# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from nltk.stem.porter import PorterStemmer
from sklearn.pipeline import FeatureUnion
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.pipeline import Pipeline
from sqlalchemy import create_engine
from nltk.corpus import stopwords
import pandas as pd
import numpy as np
import pickle
import nltk
import re

In [2]:
# Dowload several nltk packages
nltk.download(['punkt', 'wordnet', 'stopwords', 'averaged_perceptron_tagger'])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [3]:
# load data from database
engine = create_engine('sqlite:///Disaster.db')
df = pd.read_sql_table("Messages", engine)
X = df.message
cols = df.columns[4:]
Y = df[cols]

### 2. Write a tokenization function to process your text data

In [4]:
def tokenize(text):
    """Normalize the data, make a tokenNormalize, tokenize and stem text string
    
    Input:
    text: string. String that contains a message to process
       
    Output:
    normalized: list of strings. List containing a normalized and stemmed token.
    """    
    # Converting everything to lower case
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    # Tokenize words
    tokens = word_tokenize(text)
    # normalization word tokens and remove stop words
    normlizer = PorterStemmer()
    stop_words = stopwords.words("english")
    normalized = [normlizer.stem(word) for word in tokens if word not in stop_words]
    return normalized

In [5]:
tokenize(X[0])

['weather', 'updat', 'cold', 'front', 'cuba', 'could', 'pass', 'haiti']

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
# Build the first pipeline
pipe1 = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [7]:
# Divide the dataset in train and test, finally train the model
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size=0.15, random_state=42)
pipe1.fit(Xtrain, Ytrain);

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [8]:
# make predictions and get the test data and target names
predictions = pipe1.predict(Xtest)
#predictions.shape, Ytest.values.shape, len(Y.columns.values)
ground_truth = Ytest.values
target_names = Y.columns.values

In [9]:
def print_metrics(gt, pred, col_names):
    """Evaluation Metrics 
    
    Input:
    gt: array. Array of dataset ground truth.
    pred: array. Array of dataset predictions.
    col_names: strig list. Name list of predicted labels.
       
    Output:
    df: dataframe. Contains the same output as classification_report
    """
    metrics = []
    
    # Calculate evaluation metrics for each set of labels
    for i in range(len(col_names)):
        acc = accuracy_score(gt[:, i], pred[:, i])
        prec = precision_score(gt[:, i], pred[:, i], average='micro')
        rec = recall_score(gt[:, i], pred[:, i], average='micro')
        f1 = f1_score(gt[:, i], pred[:, i], average='micro')
        
        metrics.append([acc, prec, rec, f1])
    
    # Create dataframe containing metrics
    metrics = np.array(metrics)
    columns = ['Accuracy', 'Precision', 'Recall', 'F1']
    metrics_df = pd.DataFrame(data = metrics, index = col_names, columns=columns)
      
    return metrics_df

In [10]:
print_metrics(ground_truth, predictions, target_names)

,Accuracy,Precision,Recall,F1
related,0.800661,0.800661,0.800661,0.800661
request,0.884058,0.884058,0.884058,0.884058
offer,0.994152,0.994152,0.994152,0.994152
aid_related,0.744470,0.744470,0.744470,0.744470
medical_help,0.926519,0.926519,0.926519,0.926519
medical_products,0.953471,0.953471,0.953471,0.953471
search_and_rescue,0.975845,0.975845,0.975845,0.975845
security,0.982456,0.982456,0.982456,0.982456
military,0.969489,0.969489,0.969489,0.969489
child_alone,1.000000,1.000000,1.000000,1.000000


### 6. Improve your model
Use grid search to find better parameters. 

In [11]:
# Let's see the model parameters
pipe1.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7f76f94b30d0>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=None,

In [12]:
# Play with the model parameters and make a grid search
parameters = {
        'vect__ngram_range': ((1, 1), (1, 2)),
        #'vector__min_df':[1, 5],
        #'vector__max_df': (0.5, 0.75, 1.0),
        #'vector__max_features': (None, 5000, 10000),
        'tfidf__use_idf': [True, False],
        #'clf__estimator__n_estimators': [5, 10, 15],
        'clf__estimator__min_samples_split': [2, 5]
    }

cv = GridSearchCV(pipe1, param_grid=parameters, verbose=4)

In [13]:
# Train a random forest multiclass classifier
model_randomforest = cv.fit(Xtrain, Ytrain)

Fitting 3 folds for each of 8 candidates, totalling 24 fits
[CV] clf__estimator__min_samples_split=2, tfidf__use_idf=True, vect__ngram_range=(1, 1) 
[CV]  clf__estimator__min_samples_split=2, tfidf__use_idf=True, vect__ngram_range=(1, 1), score=0.23411416262789445, total= 1.1min
[CV] clf__estimator__min_samples_split=2, tfidf__use_idf=True, vect__ngram_range=(1, 1) 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.4min remaining:    0.0s


[CV]  clf__estimator__min_samples_split=2, tfidf__use_idf=True, vect__ngram_range=(1, 1), score=0.2425955842757135, total= 1.1min
[CV] clf__estimator__min_samples_split=2, tfidf__use_idf=True, vect__ngram_range=(1, 1) 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  2.9min remaining:    0.0s


[CV]  clf__estimator__min_samples_split=2, tfidf__use_idf=True, vect__ngram_range=(1, 1), score=0.24208967281540325, total= 1.1min
[CV] clf__estimator__min_samples_split=2, tfidf__use_idf=True, vect__ngram_range=(1, 2) 


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:  4.3min remaining:    0.0s


[CV]  clf__estimator__min_samples_split=2, tfidf__use_idf=True, vect__ngram_range=(1, 2), score=0.2451534733441034, total= 2.5min
[CV] clf__estimator__min_samples_split=2, tfidf__use_idf=True, vect__ngram_range=(1, 2) 
[CV]  clf__estimator__min_samples_split=2, tfidf__use_idf=True, vect__ngram_range=(1, 2), score=0.250942380183091, total= 2.6min
[CV] clf__estimator__min_samples_split=2, tfidf__use_idf=True, vect__ngram_range=(1, 2) 
[CV]  clf__estimator__min_samples_split=2, tfidf__use_idf=True, vect__ngram_range=(1, 2), score=0.24747542749427764, total= 2.5min
[CV] clf__estimator__min_samples_split=2, tfidf__use_idf=False, vect__ngram_range=(1, 1) 
[CV]  clf__estimator__min_samples_split=2, tfidf__use_idf=False, vect__ngram_range=(1, 1), score=0.24111470113085623, total= 1.1min
[CV] clf__estimator__min_samples_split=2, tfidf__use_idf=False, vect__ngram_range=(1, 1) 
[CV]  clf__estimator__min_samples_split=2, tfidf__use_idf=False, vect__ngram_range=(1, 1), score=0.23303715670436187, to

[Parallel(n_jobs=1)]: Done  24 out of  24 | elapsed: 49.1min finished


In [14]:
# Show the best model results
model_randomforest.best_params_

{'clf__estimator__min_samples_split': 2,
 'tfidf__use_idf': True,
 'vect__ngram_range': (1, 2)}

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [15]:
# Test the random forest multiclass classifier with the test data
predictions = model_randomforest.predict(Xtest)
ground_truth = Ytest.values
target_names = Ytest.columns.values

print_metrics(ground_truth, predictions, target_names)

,Accuracy,Precision,Recall,F1
related,0.791508,0.791508,0.791508,0.791508
request,0.883804,0.883804,0.883804,0.883804
offer,0.994152,0.994152,0.994152,0.994152
aid_related,0.743199,0.743199,0.743199,0.743199
medical_help,0.922197,0.922197,0.922197,0.922197
medical_products,0.951437,0.951437,0.951437,0.951437
search_and_rescue,0.977371,0.977371,0.977371,0.977371
security,0.982202,0.982202,0.982202,0.982202
military,0.966946,0.966946,0.966946,0.966946
child_alone,1.000000,1.000000,1.000000,1.000000


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [16]:
# Prepare the next pipeline with an Adaboost Multiclass Classifier
pipe2 = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(AdaBoostClassifier()))
])

In [17]:
# Get the parameters of this model
pipe2.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7f76f94b30d0>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
             learning_rate=1.0, n_estimators=50, random_state=None),
              n_jobs=1))],
 'vect': CountVectorizer(analyzer='word', binary=False, decode_error='strict',
         dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
         lowercase=True, max_df=1.0, max_features=None, min_df=1,
         ngram_rang

In [22]:
# get the parameters and made a grid search, finally train the adaboost multiclass classifier
parameters = {
        'vect__ngram_range': ((1,1), (1,2)),
        'tfidf__use_idf': [True, False],
        'clf__estimator__n_estimators': [10, 25, 50],
    }

cv = GridSearchCV(pipe2, param_grid=parameters, verbose=4)
model_adaboost = cv.fit(Xtrain, Ytrain)

Fitting 3 folds for each of 12 candidates, totalling 36 fits
[CV] clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__ngram_range=(1, 1) 
[CV]  clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__ngram_range=(1, 1), score=0.1894184168012924, total=  39.9s
[CV] clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__ngram_range=(1, 1) 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   57.7s remaining:    0.0s


[CV]  clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__ngram_range=(1, 1), score=0.19978459881529348, total=  40.5s
[CV] clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__ngram_range=(1, 1) 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  1.9min remaining:    0.0s


[CV]  clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__ngram_range=(1, 1), score=0.19671468964588662, total=  40.8s
[CV] clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__ngram_range=(1, 2) 


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:  2.9min remaining:    0.0s


[CV]  clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__ngram_range=(1, 2), score=0.19116855142703285, total= 1.2min
[CV] clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__ngram_range=(1, 2) 
[CV]  clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__ngram_range=(1, 2), score=0.19547657512116318, total= 1.2min
[CV] clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__ngram_range=(1, 2) 
[CV]  clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__ngram_range=(1, 2), score=0.19065571563215294, total= 1.2min
[CV] clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__ngram_range=(1, 1) 
[CV]  clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__ngram_range=(1, 1), score=0.19184168012924072, total=  38.8s
[CV] clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__ngram_range=(1, 1) 
[CV]  clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__ngram_range=(1, 1), score=0.20180398492191706, total=  38.7s
[CV] clf__estimator__

[Parallel(n_jobs=1)]: Done  36 out of  36 | elapsed: 76.2min finished


In [23]:
# Show the best parameters
model_adaboost.best_params_

{'clf__estimator__n_estimators': 50,
 'tfidf__use_idf': True,
 'vect__ngram_range': (1, 1)}

In [24]:
# make the predictions using adaboost
predictions = model_adaboost.predict(Xtest)
ground_truth = Ytest.values
target_names = Ytest.columns.values

print_metrics(ground_truth, predictions, target_names)

,Accuracy,Precision,Recall,F1
related,0.769641,0.769641,0.769641,0.769641
request,0.887109,0.887109,0.887109,0.887109
offer,0.993135,0.993135,0.993135,0.993135
aid_related,0.754132,0.754132,0.754132,0.754132
medical_help,0.925502,0.925502,0.925502,0.925502
medical_products,0.956776,0.956776,0.956776,0.956776
search_and_rescue,0.977117,0.977117,0.977117,0.977117
security,0.980676,0.980676,0.980676,0.980676
military,0.968980,0.968980,0.968980,0.968980
child_alone,1.000000,1.000000,1.000000,1.000000


### 9. Export your model as a pickle file

In [25]:
# Save the best version of the model, this is the case for the random forest
filename = 'Response.pkl'
pickle.dump(model_randomforest, open(filename, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

In [26]:
# import libraries
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from nltk.stem.porter import PorterStemmer
from sklearn.pipeline import FeatureUnion
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.pipeline import Pipeline
from sqlalchemy import create_engine
from nltk.corpus import stopwords
import pandas as pd
import numpy as np
import pickle
import nltk
import re

In [42]:
def load_data(database_filepath):
    """Load the database to train
    
    Input:
    database_filepath: string. String that contains a database file to read containing the messages
       
    Output:
    X: series.  A set of messages of the database to train
    Y: series.  A set of labels of the database to train
    category_names: array. List of the 36 classes to classify.
    """    
    nltk.download(['punkt', 'wordnet', 'stopwords', 'averaged_perceptron_tagger'])
    engine = create_engine('sqlite:///' + database_filepath)
    df = pd.read_sql_table("Messages", engine)
    X = df.message
    category_names = df.columns[4:]
    Y = df[category_names]
    return X, Y, category_names
        
def tokenize(text):
    """Normalize the data, make a tokenNormalize, tokenize and stem text string
    
    Input:
    text: string. String that contains a message to process
       
    Output:
    normalized: list of strings. List containing a normalized and stemmed token.
    """    
    # Converting everything to lower case
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    # Tokenize words
    tokens = word_tokenize(text)
    # normalization word tokens and remove stop words
    normlizer = PorterStemmer()
    stop_words = stopwords.words("english")
    normalized = [normlizer.stem(word) for word in tokens if word not in stop_words]
    return normalized
    
def build_model():
    """Make the model and do a grid search
    
    Input:
    None
       
    Output:
    model:  The resulting model before the training process, gridsearch set.
    """    
    pipe = Pipeline([
        ('vect', CountVectorizer(tokenizer = tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])
  
    parameters = {
        'vect__ngram_range': ((1,1),(1, 2)),
        'tfidf__use_idf': [True],
        'clf__estimator__min_samples_split': [2, 4]
    }    
    
    model = GridSearchCV(pipe, param_grid=parameters, verbose=4)
    return model


def __print_metrics(gt, pred, col_names):
    """Evaluation Metrics 
    
    Input:
    gt: array. Array of dataset ground truth.
    pred: array. Array of dataset predictions.
    col_names: strig list. Name list of predicted labels.
       
    Output:
    df: dataframe. Contains the same output as classification_report
    """
    metrics = []
    
    # Calculate evaluation metrics for each set of labels
    for i in range(len(col_names)):
        acc = accuracy_score(gt[:, i], pred[:, i])
        prec = precision_score(gt[:, i], pred[:, i], average='micro')
        rec = recall_score(gt[:, i], pred[:, i], average='micro')
        f1 = f1_score(gt[:, i], pred[:, i], average='micro')
        
        metrics.append([acc, prec, rec, f1])
    
    # Create dataframe containing metrics
    metrics = np.array(metrics)
    columns = ['Accuracy', 'Precision', 'Recall', 'F1']
    metrics_df = pd.DataFrame(data = metrics, index = col_names, columns=columns)
      
    return metrics_df


def evaluate_model(model, X_test, Y_test, category_names):
    """Predict and evaluate some metrics of the model
    
    Input:
    model:  The model of the current data to evaluate
    X_test:  series.  The feature values of the test set to evaluate
    Y_test:  series.  The label values of the dataset to evaluate 
    category_names:  list of strings. The array of the labels to evaluate
       
    Output:
    None.  But it displays the printed output of the metrics of the model
    """    
    predictions = model.predict(X_test)
    ground_truth = Y_test.values
    target_names = category_names #Ytest.columns.values.tolist()
    print(__print_metrics(ground_truth, predictions, target_names))
    
def save_model(model, model_filepath):
    """Save the final model of the disaster response dataset
    
    Input:
    model: The model to save
    model_filepath:  string.  The path of the model to save
       
    Output:
    None
    """    
    pickle.dump(model, open(model_filepath, 'wb'))

In [44]:
# Proceed to load the data, split, build the model and fit it
database_filepath, model_filepath = 'Disaster.db', 'Response.pkl'
print('Loading data...\n    DATABASE: {}'.format(database_filepath))

X, Y, category_names = load_data(database_filepath)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.15)

print('Building model...')
model = build_model()

print('Training model...')
model.fit(X_train, Y_train)

Loading data...
    DATABASE: Disaster.db
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
Building model...
Training model...
Fitting 3 folds for each of 4 candidates, totalling 12 fits
[CV] clf__estimator__min_samples_split=2, tfidf__use_idf=True, vect__ngram_range=(1, 1) 
[CV]  clf__estimator__min_samples_split=2, tfidf__use_idf=True, vect__ngram_range=(1, 1), score=0.23128702207862142, total= 1.1min
[CV] clf__estimator__min_samples_split=2, tfidf__use_idf=True, vect__ngram_range=(1, 1) 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.4min remaining:    0.0s


[CV]  clf__estimator__min_samples_split=2, tfidf__use_idf=True, vect__ngram_range=(1, 1), score=0.2460958535271944, total= 1.1min
[CV] clf__estimator__min_samples_split=2, tfidf__use_idf=True, vect__ngram_range=(1, 1) 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  2.9min remaining:    0.0s


[CV]  clf__estimator__min_samples_split=2, tfidf__use_idf=True, vect__ngram_range=(1, 1), score=0.23441497239800727, total= 1.1min
[CV] clf__estimator__min_samples_split=2, tfidf__use_idf=True, vect__ngram_range=(1, 2) 


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:  4.3min remaining:    0.0s


[CV]  clf__estimator__min_samples_split=2, tfidf__use_idf=True, vect__ngram_range=(1, 2), score=0.23963381798599892, total= 2.6min
[CV] clf__estimator__min_samples_split=2, tfidf__use_idf=True, vect__ngram_range=(1, 2) 
[CV]  clf__estimator__min_samples_split=2, tfidf__use_idf=True, vect__ngram_range=(1, 2), score=0.24138395261173937, total= 2.6min
[CV] clf__estimator__min_samples_split=2, tfidf__use_idf=True, vect__ngram_range=(1, 2) 
[CV]  clf__estimator__min_samples_split=2, tfidf__use_idf=True, vect__ngram_range=(1, 2), score=0.2412818096135721, total= 2.6min
[CV] clf__estimator__min_samples_split=4, tfidf__use_idf=True, vect__ngram_range=(1, 1) 
[CV]  clf__estimator__min_samples_split=4, tfidf__use_idf=True, vect__ngram_range=(1, 1), score=0.2358642972536349, total=  59.8s
[CV] clf__estimator__min_samples_split=4, tfidf__use_idf=True, vect__ngram_range=(1, 1) 
[CV]  clf__estimator__min_samples_split=4, tfidf__use_idf=True, vect__ngram_range=(1, 1), score=0.23397953688745288, total

[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed: 24.7min finished


GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'vect__ngram_range': ((1, 1), (1, 2)), 'tfidf__use_idf': [True], 'clf__estimator__min_samples_split': [2, 4]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=4)

In [45]:
# Proceed to make evaluaton metrics, save the model and finish
print('Evaluating model...')
evaluate_model(model, X_test, Y_test, category_names)

print('Saving model...\n    MODEL: {}'.format(model_filepath))
save_model(model, model_filepath)

print('Trained model saved!')

Evaluating model...
                        Accuracy  Precision    Recall        F1
related                 0.799644   0.799644  0.799644  0.799644
request                 0.887618   0.887618  0.887618  0.887618
offer                   0.993898   0.993898  0.993898  0.993898
aid_related             0.747267   0.747267  0.747267  0.747267
medical_help            0.913298   0.913298  0.913298  0.913298
medical_products        0.950928   0.950928  0.950928  0.950928
search_and_rescue       0.968218   0.968218  0.968218  0.968218
security                0.979405   0.979405  0.979405  0.979405
military                0.965929   0.965929  0.965929  0.965929
child_alone             1.000000   1.000000  1.000000  1.000000
water                   0.951182   0.951182  0.951182  0.951182
food                    0.927282   0.927282  0.927282  0.927282
shelter                 0.928808   0.928808  0.928808  0.928808
clothing                0.984490   0.984490  0.984490  0.984490
money               